<a href="https://colab.research.google.com/github/Wency-WangC/Nice-Little-Tools/blob/main/converter_v2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
### Lunar Date Converter with Dual Input Modes for Colab
# 安装依赖
!pip install -q opencc-python-reimplemented pandas openpyxl ipywidgets

from google.colab import files
import pandas as pd
import io, os
from opencc import OpenCC
import ipywidgets as widgets
from IPython.display import display, clear_output

# 简体转繁体引擎
cc = OpenCC('s2t')

# 全局存储列表
entries = []

# 核心导出函数

def convert_and_download(df: pd.DataFrame, filename: str = "output_input.csv"):
    # 自动补齐字段
    for col in ["国号", "帝号"]:
        if col not in df.columns:
            df[col] = ""
    df = df[["国号", "帝号", "年号", "年", "月", "日"]]
    df = df.fillna("")
    # 简转繁
    df = df.applymap(lambda x: cc.convert(str(x)) if isinstance(x, str) else x)
    # 导出 CSV
    df.to_csv(filename, index=False, header=False, encoding="utf-8")
    files.download(filename)
    print(f"✅ 下载完成：{filename}")

# Excel 上传分支

def upload_excel_ui():
    uploaded = files.upload()
    fname = list(uploaded.keys())[0]
    df = pd.read_excel(io.BytesIO(uploaded[fname]), dtype=str)
    print("✅ 上传并读取完成，前几行：")
    display(df.head())
    convert_and_download(df, f"{os.path.splitext(fname)[0]}_input.csv")

# 手动输入分支：单条录入

def manual_input_ui():
    clear_output()
    # 字段控件
    field_guo = widgets.Text(description='国号：')
    field_di = widgets.Text(description='帝号：')
    field_era = widgets.Text(description='年号：')
    field_year = widgets.BoundedIntText(value=1, min=1, max=9999, description='年：')
    field_month = widgets.Text(description='月：', placeholder='可输入“3”或“闰7”')
    field_day = widgets.BoundedIntText(value=1, min=1, max=31, description='日：')

    # 按钮与输出
    next_btn = widgets.Button(description='下一条')
    submit_btn = widgets.Button(description='提交并处理')
    manual_out = widgets.Output()

    def on_next(b):
        with manual_out:
            # 组装记录
            rec = {
                '国号': field_guo.value,
                '帝号': field_di.value,
                '年号': field_era.value,
                '年': str(field_year.value),
                '月': field_month.value,
                '日': str(field_day.value)
            }
            entries.append(rec)
            # 打印友好提示
            label = f"✅ 已添加{rec['国号']}{rec['帝号']}{rec['年号']}{rec['年']}年{rec['月']}月{rec['日']}日"
            print(label)
            # 清空输入
            field_guo.value = ''
            field_di.value = ''
            field_era.value = ''
            field_year.value = 1
            field_month.value = ''
            field_day.value = 1

    def on_submit(b):
        with manual_out:
            if not entries:
                print("❌ 请先点击“下一条”添加至少一条记录。")
                return
            df = pd.DataFrame(entries)
            print("✅ 所有记录：")
            display(df)
            convert_and_download(df, 'manual_input.csv')

    next_btn.on_click(on_next)
    submit_btn.on_click(on_submit)

    # 渲染控件
    display(
        widgets.VBox([
            widgets.HBox([field_guo, field_di]),
            widgets.HBox([field_era, field_year]),
            widgets.HBox([field_month, field_day]),
            widgets.HBox([next_btn, submit_btn]),
            manual_out
        ])
    )

# 主界面：模式选择 + 确定按钮
mode_selector = widgets.RadioButtons(
    options=[('上传 Excel 文件', 'excel'), ('手动输入/粘贴中历日期', 'manual')],
    description='选择模式：'
)
confirm_button = widgets.Button(description='确定')
ui_output = widgets.Output()

# 确定按钮回调

def on_confirm(b):
    ui_output.clear_output()
    with ui_output:
        if mode_selector.value == 'excel':
            upload_excel_ui()
        else:
            manual_input_ui()

confirm_button.on_click(on_confirm)

display(mode_selector, confirm_button, ui_output)
